In [1]:
from IPython.display import HTML
import numpy as np
import urllib3
from bs4 import BeautifulSoup as soup 
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

# Part I - Extraction of data from MTA's website
## Let's scraping the MTA website to get all the URLs that have weekly turnstile data in txt format

In [2]:
url = 'http://web.mta.info/developers/turnstile.html'
manager = urllib3.PoolManager()
response = manager.request('GET', url)

In [3]:
html = soup(response.data, 'html.parser')
html


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN"
   "http://www.w3.org/TR/html4/strict.dtd">

<html lang="en">
<head>
<title>mta.info | Turnstile Data</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!--<meta http-equiv="X-UA-Compatible" content="IE=EmulateIE7">-->
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="/siteimages/favicon.ico" rel="shortcut icon"/>
<link href="/css/base.css" rel="stylesheet" type="text/css"/>
<link href="/css/grid.css" rel="stylesheet" type="text/css"/>
<link href="/css/topbar.css" rel="stylesheet" type="text/css"/>
<link href="/css/formalize.css" rel="stylesheet" type="text/css"/>
<!-- <link rel="stylesheet" type="text/css" href="/css/jquery.datepick.css"> -->
<!-- jQuery include should be at the top -->
<script language="javascript" src="/js/jquery-1.4.4.min.js" type="text/javascript"></script>
<link href="/css/template.css" rel="stylesheet" type="text/css"/>
<meta content="News" name="WT.cg_n">
<style media

In [4]:
html.head.title

<title>mta.info | Turnstile Data</title>

In [5]:
data_files = html.find_all('div', {'class':'span-84 last'})

In [6]:
len(data_files)

1

In [7]:
data_files[0]

<div class="span-84 last">
<h2>Data Files</h2>
<br/>
<a href="data/nyct/turnstile/turnstile_180310.txt">Saturday, March 10, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180303.txt">Saturday, March 03, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180224.txt">Saturday, February 24, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180217.txt">Saturday, February 17, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180210.txt">Saturday, February 10, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180203.txt">Saturday, February 03, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180127.txt">Saturday, January 27, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180120.txt">Saturday, January 20, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180113.txt">Saturday, January 13, 2018</a><br/><a href="data/nyct/turnstile/turnstile_180106.txt">Saturday, January 06, 2018</a><br/><a href="data/nyct/turnstile/turnstile_171230.txt">Saturday, December 30, 2017</a><br/>

In [8]:
data_files[0].find_all('a')

[<a href="data/nyct/turnstile/turnstile_180310.txt">Saturday, March 10, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180303.txt">Saturday, March 03, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180224.txt">Saturday, February 24, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180217.txt">Saturday, February 17, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180210.txt">Saturday, February 10, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180203.txt">Saturday, February 03, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180127.txt">Saturday, January 27, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180120.txt">Saturday, January 20, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180113.txt">Saturday, January 13, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180106.txt">Saturday, January 06, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_171230.txt">Saturday, December 30, 2017</a>,
 <a href="data/nyct/turnstile/turnstile_171223.txt">Saturday, December 23, 

#### Store all the URLs in list

In [9]:
list_of_urls = []
for elem in data_files[0].find_all('a'):
    list_of_urls.append(elem.get('href'))

In [10]:
list_of_urls

['data/nyct/turnstile/turnstile_180310.txt',
 'data/nyct/turnstile/turnstile_180303.txt',
 'data/nyct/turnstile/turnstile_180224.txt',
 'data/nyct/turnstile/turnstile_180217.txt',
 'data/nyct/turnstile/turnstile_180210.txt',
 'data/nyct/turnstile/turnstile_180203.txt',
 'data/nyct/turnstile/turnstile_180127.txt',
 'data/nyct/turnstile/turnstile_180120.txt',
 'data/nyct/turnstile/turnstile_180113.txt',
 'data/nyct/turnstile/turnstile_180106.txt',
 'data/nyct/turnstile/turnstile_171230.txt',
 'data/nyct/turnstile/turnstile_171223.txt',
 'data/nyct/turnstile/turnstile_171216.txt',
 'data/nyct/turnstile/turnstile_171209.txt',
 'data/nyct/turnstile/turnstile_171202.txt',
 'data/nyct/turnstile/turnstile_171125.txt',
 'data/nyct/turnstile/turnstile_171118.txt',
 'data/nyct/turnstile/turnstile_171111.txt',
 'data/nyct/turnstile/turnstile_171104.txt',
 'data/nyct/turnstile/turnstile_171028.txt',
 'data/nyct/turnstile/turnstile_171021.txt',
 'data/nyct/turnstile/turnstile_171014.txt',
 'data/nyc

## Time to create a dataframe using above URLs
### Our analysis is only going to focus on year 2017. So we need to get the URLs for year 2017 and create a DataFrame

In [15]:
print(list_of_urls[0], "\nlength is", len(list_of_urls[0]))

data/nyct/turnstile/turnstile_180310.txt 
length is 40


In [20]:
list_of_urls[0][30:32]

'18'

#### So we need to grab the characters at index 30 & 31 to get the year

In [23]:
year_2017 = list()

In [26]:
for item in list_of_urls:
    if item[30:32] == '17':
        year_2017.append(item)


In [28]:
len(year_2017)

52

In [29]:
base_url = "http://web.mta.info/developers/"
base_url+year_2017[0]

'http://web.mta.info/developers/data/nyct/turnstile/turnstile_171230.txt'

In [32]:
mta = pd.DataFrame()
countdown = len(year_2017) + 1

for i in year_2017:
    temp = pd.read_csv(base_url+i)
    mta = mta.append(temp, ignore_index=True) #appending will result in index getting repeated and hence they need to be ignored
    countdown = countdown - 1
    print("Done with week",countdown)

Done with week 52
Done with week 51
Done with week 50
Done with week 49
Done with week 48
Done with week 47
Done with week 46
Done with week 45
Done with week 44
Done with week 43
Done with week 42
Done with week 41
Done with week 40
Done with week 39
Done with week 38
Done with week 37
Done with week 36
Done with week 35
Done with week 34
Done with week 33
Done with week 32
Done with week 31
Done with week 30
Done with week 29
Done with week 28
Done with week 27
Done with week 26
Done with week 25
Done with week 24
Done with week 23
Done with week 22
Done with week 21
Done with week 20
Done with week 19
Done with week 18
Done with week 17
Done with week 16
Done with week 15
Done with week 14
Done with week 13
Done with week 12
Done with week 11
Done with week 10
Done with week 9
Done with week 8
Done with week 7
Done with week 6
Done with week 5
Done with week 4
Done with week 3
Done with week 2
Done with week 1


In [33]:
mta.shape

(10258959, 11)

In [34]:
mta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10258959 entries, 0 to 10258958
Data columns (total 11 columns):
C/A                                                                     object
UNIT                                                                    object
SCP                                                                     object
STATION                                                                 object
LINENAME                                                                object
DIVISION                                                                object
DATE                                                                    object
TIME                                                                    object
DESC                                                                    object
ENTRIES                                                                 int64
EXITS                                                                   int64
dtypes: int64(2), 

We need to change DATE attribute's datatype to date time

In [ ]:
mta["DATE"] = pd.to_datetime(mta["DATE"])

In [35]:
mta.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/23/2017,03:00:00,REGULAR,6455840,2184987
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/23/2017,07:00:00,REGULAR,6455856,2184995
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/23/2017,11:00:00,REGULAR,6455899,2185082
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/23/2017,15:00:00,REGULAR,6456038,2185156
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/23/2017,19:00:00,REGULAR,6456327,2185197


In [36]:
mta.tail()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
10258954,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/06/2017,04:00:00,REGULAR,5554,283
10258955,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/06/2017,08:00:00,REGULAR,5554,283
10258956,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/06/2017,12:00:00,REGULAR,5554,283
10258957,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/06/2017,16:00:00,REGULAR,5554,283
10258958,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/06/2017,20:00:00,REGULAR,5554,283


In [37]:
mta.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

### So there are trailing spaces in EXITS column... We'll rename it.

In [38]:
mta.rename(columns={'EXITS                                                               ':'EXITS'}, inplace=True)

In [39]:
mta.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [41]:
len(mta["DATE"].unique())

364

### So apart from Jan & Feb 2018, we have 31/12/2016 as well in the dataset. So let's select year 2017 only for the analysis.

In [20]:
test = mta[mta.indexex < 10]

In [21]:
test

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/17/2018,03:00:00,REGULAR,6522160,2208760
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/17/2018,07:00:00,REGULAR,6522160,2208760
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/17/2018,11:00:00,REGULAR,6522161,2208760
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/17/2018,15:00:00,REGULAR,6522164,2208762
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/17/2018,19:00:00,REGULAR,6522164,2208762
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/17/2018,23:00:00,REGULAR,6522164,2208762
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/18/2018,03:00:00,REGULAR,6522164,2208762
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/18/2018,07:00:00,REGULAR,6522164,2208762
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/18/2018,11:00:00,REGULAR,6522164,2208762
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/18/2018,15:00:00,REGULAR,6522164,2208762


In [22]:
pd.to_datetime(test["DATE"])

0   2018-02-17
1   2018-02-17
2   2018-02-17
3   2018-02-17
4   2018-02-17
5   2018-02-17
6   2018-02-18
7   2018-02-18
8   2018-02-18
9   2018-02-18
Name: DATE, dtype: datetime64[ns]

In [23]:
test['DATE'] = pd.to_datetime(test["DATE"])

/Users/akshayjam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 11 columns):
C/A                                                                     10 non-null object
UNIT                                                                    10 non-null object
SCP                                                                     10 non-null object
STATION                                                                 10 non-null object
LINENAME                                                                10 non-null object
DIVISION                                                                10 non-null object
DATE                                                                    10 non-null datetime64[ns]
TIME                                                                    10 non-null object
DESC                                                                    10 non-null object
ENTRIES                                                                

In [25]:
test.dtypes

C/A                                                                             object
UNIT                                                                            object
SCP                                                                             object
STATION                                                                         object
LINENAME                                                                        object
DIVISION                                                                        object
DATE                                                                    datetime64[ns]
TIME                                                                            object
DESC                                                                            object
ENTRIES                                                                          int64
EXITS                                                                            int64
dtype: object

In [26]:
test.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-02-17,03:00:00,REGULAR,6522160,2208760
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-02-17,07:00:00,REGULAR,6522160,2208760
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-02-17,11:00:00,REGULAR,6522161,2208760
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-02-17,15:00:00,REGULAR,6522164,2208762
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-02-17,19:00:00,REGULAR,6522164,2208762


In [28]:
test["DATE_TIME"] = test["DATE"].map(str) + test["TIME"]

/Users/akshayjam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
mta['DATE'] = pd.to_datetime(mta["DATE"])

In [26]:
mta.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-02-17,03:00:00,REGULAR,6522160,2208760
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-02-17,07:00:00,REGULAR,6522160,2208760
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-02-17,11:00:00,REGULAR,6522161,2208760
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-02-17,15:00:00,REGULAR,6522164,2208762
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-02-17,19:00:00,REGULAR,6522164,2208762


In [27]:
mta.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [28]:
data = mta[(mta["DATE"] > "2016-12-31") & (mta["DATE"] < "2018-01-01")]

In [29]:
min(data["DATE"])

Timestamp('2017-01-01 00:00:00')

In [30]:
max(data["DATE"])

Timestamp('2017-12-31 00:00:00')

In [31]:
data.shape

(10287525, 11)

### So now we have the data for year 2017, we are ready for the analysis part.

# Part II - Analysis
#### Some details about the attributes

C/A      = Control Area (A002)

UNIT     = Remote Unit for a station (R051)

SCP      = Subunit Channel Position represents an specific address for a device (02-00-00)

STATION  = Represents the station name the device is located at

LINENAME = Represents all train lines that can be boarded at this station
                   Normally lines are represented by one character.  LINENAME 456NQR repersents train server for 4, 5, 6, N,Q, and R trains.

DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND   

DATE     = Represents the date (MM-DD-YY)

TIME     = Represents the time (hh:mm:ss) for a scheduled audit event

DESc     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
           1. Audits may occur more that 4 hours due to planning, or troubleshooting activities. 
           2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered. 

ENTRIES  = The comulative entry register value for a device

EXIST    = The cumulative exit register value for a device


## Which station has the most number of units?

In [32]:
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
1395663,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,03:00:00,REGULAR,6463109,2187558
1395664,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,07:00:00,REGULAR,6463119,2187567
1395665,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,11:00:00,REGULAR,6463178,2187654
1395666,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,15:00:00,REGULAR,6463373,2187736
1395667,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,19:00:00,REGULAR,6463712,2187803


In [33]:
data.groupby("STATION").UNIT.count().sort_values(ascending=False)[:5]

STATION
34 ST-PENN STA     218003
FULTON ST          211555
23 ST              155923
GRD CNTRL-42 ST    142021
CANAL ST           128581
Name: UNIT, dtype: int64

##### So 34th Street - Penn Station has the maximum number of units i.e. turnstiles. This means that it should be the most crowded and busy station. Let's verify that by answering the below question.

## What is the total number of entries & exits across the subway system for August 1, 2017?

In [34]:
data['Busyness'] = data["ENTRIES"] + data["EXITS"]

/Users/akshayjam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10287525 entries, 1395663 to 11855286
Data columns (total 12 columns):
C/A         object
UNIT        object
SCP         object
STATION     object
LINENAME    object
DIVISION    object
DATE        datetime64[ns]
TIME        object
DESC        object
ENTRIES     int64
EXITS       int64
Busyness    int64
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 1020.3+ MB


In [36]:
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Busyness
1395663,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,03:00:00,REGULAR,6463109,2187558,8650667
1395664,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,07:00:00,REGULAR,6463119,2187567,8650686
1395665,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,11:00:00,REGULAR,6463178,2187654,8650832
1395666,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,15:00:00,REGULAR,6463373,2187736,8651109
1395667,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,19:00:00,REGULAR,6463712,2187803,8651515


In [71]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10287525 entries, 1395663 to 11855286
Data columns (total 11 columns):
C/A                                                                     object
UNIT                                                                    object
SCP                                                                     object
STATION                                                                 object
LINENAME                                                                object
DIVISION                                                                object
DATE                                                                    datetime64[ns]
TIME                                                                    object
DESC                                                                    object
ENTRIES                                                                 int64
EXITS                                                                   int64
dtyp

In [72]:
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
1395663,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,03:00:00,REGULAR,6463109,2187558
1395664,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,07:00:00,REGULAR,6463119,2187567
1395665,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,11:00:00,REGULAR,6463178,2187654
1395666,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,15:00:00,REGULAR,6463373,2187736
1395667,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,19:00:00,REGULAR,6463712,2187803


In [80]:
mta['EXITS'].isnull()

KeyError: 'EXITS'

In [88]:
data.iloc[:,[10]]

,EXITS
1395663,2187558
1395664,2187567
1395665,2187654
1395666,2187736
1395667,2187803
1395668,2187833
1395669,2187844
1395670,2187856
1395671,2187919
1395672,2187969


In [91]:
data['new_Column'] = data["ENTRIES"] + data["ENTRIES"]

/Users/akshayjam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [93]:
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,new_Column
1395663,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,03:00:00,REGULAR,6463109,2187558,12926218
1395664,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,07:00:00,REGULAR,6463119,2187567,12926238
1395665,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,11:00:00,REGULAR,6463178,2187654,12926356
1395666,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,15:00:00,REGULAR,6463373,2187736,12926746
1395667,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-30,19:00:00,REGULAR,6463712,2187803,12927424


In [94]:
data["Busyness"] = data["ENTRIES"] + data["EXITS"]

KeyError: 'EXITS'

In [2]:
mta.columns

NameError: name 'mta' is not defined

### So we need to correct this... EXITS has a lot of trailing spaces

In [ ]:
data.rename(columns={'EXITS                                                               ':'EXITS'})